In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import machado

In [1]:
import os
# Create the directory if it doesn't exist
directory = '../data/machado'
if not os.path.exists(directory):
    os.makedirs(directory)

In [3]:
# Save all texts from Machado corpus
for fileid in machado.fileids():

    dirr = fileid.split("/")[0]
    if not os.path.exists(directory+ "/" + dirr):
        os.makedirs(directory+ "/" + dirr)
    with open(os.path.join(directory, fileid), 'w', encoding='utf-8') as file:
        file.write(machado.raw(fileid))

In [4]:
sample = "../data/machado/critica/mact13.txt"
# Read the contents of the file
with open(sample, 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into paragraphs
paragraphs = text.split('\n\n')  # Assuming paragraphs are separated by two newline characters

In [5]:
paragraphs[:10]

['Crítica, O ideal do crítico, 1865',
 'O ideal do crítico',
 'Texto-Fonte:',
 'Obra Completa de Machado de Assis,',
 'Rio\nde Janeiro: Nova Aguilar, vol. III, 1994.',
 'Publicado\noriginalmente no Diário do Rio de Janeiro, 8/10/1865.',
 'Exercer a crítica, afigura-se a\nalguns que é uma fácil tarefa, como a outros parece igualmente fácil a tarefa\ndo legislador; mas, para a representação literária, como para a representação política,\né preciso ter alguma coisa mais que um simples desejo de falar à multidão.\nInfelizmente é a opinião contrária que domina, e a crítica, desamparada pelos\nesclarecidos, é exercida pelos incompetentes.',
 'São\nóbvias as conseqüências de uma tal situação. As musas, privadas de um farol\nseguro, correm o risco de naufragar nos mares sempre desconhecidos da\npublicidade. O erro produzirá o erro; amortecidos os nobres estímulos, abatidas\nas legítimas ambições, só um tribunal será acatado, e esse, se é o mais\nnumeroso, é também o menos decisivo. O poeta osc

In [6]:
import re
"""
Count the number of sentences in a paragraph.

Explanation of the regular expression used:
- (?<!\w\.\w.): Negative lookbehind assertion to ensure not matching periods within abbreviations.
- (?<![A-Z][a-z]\.): Negative lookbehind assertion to avoid matching periods in titles or abbreviations.
- (?<=\.|\?): Positive lookbehind assertion for matching periods or question marks marking the end of a sentence.
- \s: Matches any whitespace character after a period or question mark.
"""
# Regular expression pattern for splitting sentences
sentence_pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'

def count_sentences(paragraph):
    # Split the paragraph into sentences using regex
    sentences = re.split(sentence_pattern, paragraph)
    # Return the count of sentences
    return len(sentences)
    
filtered_paragraphs = [paragraph for paragraph in paragraphs if count_sentences(paragraph) > 2]
filtered_paragraphs = [paragraph.replace("\n", " ") for paragraph in filtered_paragraphs]

In [7]:
filtered_paragraphs

['São óbvias as conseqüências de uma tal situação. As musas, privadas de um farol seguro, correm o risco de naufragar nos mares sempre desconhecidos da publicidade. O erro produzirá o erro; amortecidos os nobres estímulos, abatidas as legítimas ambições, só um tribunal será acatado, e esse, se é o mais numeroso, é também o menos decisivo. O poeta oscilará entre as sentenças mal concebidas do crítico, e os arestos caprichosos da opinião; nenhuma luz, nenhum conselho, nada lhe mostrará o caminho que deve seguir, \x97 e a morte próxima será o prêmio definitivo das suas fadigas e das suas lutas.',
 'Chegamos já a estas tristes conseqüências? Não quero proferir juízo, que seria temerário, mas qualquer pode notar com que largos intervalos aparecem as boas obras, e como são raras as publicações seladas por um talento verdadeiro. Quereis mudar esta situação aflitiva? Estabelecei a crítica, mas a crítica fecunda, e não a estéril, que nos aborrece e nos mata, que não reflete nem discute, que aba

In [8]:
re.split(sentence_pattern, filtered_paragraphs[0])

['São óbvias as conseqüências de uma tal situação.',
 'As musas, privadas de um farol seguro, correm o risco de naufragar nos mares sempre desconhecidos da publicidade.',
 'O erro produzirá o erro; amortecidos os nobres estímulos, abatidas as legítimas ambições, só um tribunal será acatado, e esse, se é o mais numeroso, é também o menos decisivo.',
 'O poeta oscilará entre as sentenças mal concebidas do crítico, e os arestos caprichosos da opinião; nenhuma luz, nenhum conselho, nada lhe mostrará o caminho que deve seguir, \x97 e a morte próxima será o prêmio definitivo das suas fadigas e das suas lutas.']

In [9]:
def generate_pos_samples(filtered_paragraphs):
    df = pd.DataFrame(columns=["Context","Next_Sentence","is_same_paragraph"])
    for paragraph in filtered_paragraphs:
        sentences = re.split(sentence_pattern,paragraph)
        for i,sentence in enumerate(sentences):
            if(not len(sentence)):
                continue
            c = "[CLS] " + "".join(sentences[:i] +  sentences[i+1:])
            s = "[CLS] "  + sentence
            df = pd.concat([pd.DataFrame([[c,s,1]], columns=df.columns), df], ignore_index=True)
    
        for i in range(1,len(sentences)):
            sentence = sentences[i]

            if(not len(sentence)):
                continue

            c = "[CLS] " + sentences[i-1]
            s = "[CLS] "  + sentences[i]
            df = pd.concat([pd.DataFrame([[c,s,1]], columns=df.columns), df], ignore_index=True)
    return df

In [10]:
def generate_neg_samples(filtered_paragraphs,n_samples):
    df = pd.DataFrame(columns=["Context","Next_Sentence","is_same_paragraph"])
    for _ in range(n_samples):
        try: 
            context_size = np.random.randint(5) + 1
            random_paragraphs = np.random.randint(0,len(filtered_paragraphs),context_size)
            random_paragraphs = [filtered_paragraphs[i] for i in random_paragraphs]
        
            for paragraph_1 in random_paragraphs:
                for paragraph_2 in random_paragraphs:
                    if(paragraph_1 == paragraph_2):
                        break
                
                    sentences_2 = re.split(sentence_pattern,paragraph_2)
                    sentences_2 = [p for p in sentences_2 if len(p)]

                    index  = np.random.randint(0,len(sentences_2))

                    c = "[CLS] " + paragraph_1
                    s = "[CLS] " + sentences_2[index]
                    df = pd.concat([pd.DataFrame([[c,s,0]], columns=df.columns), df], ignore_index=True)
                

            for i in range(1,context_size):
                paragraph_1 = re.split(sentence_pattern,random_paragraphs[i-1])
                paragraph_2 = re.split(sentence_pattern,random_paragraphs[i])

                paragraph_1 = [p for p in paragraph_1 if len(p)]
                paragraph_2 = [p for p in paragraph_2 if len(p)]

                first_index  = np.random.randint(0,len(paragraph_1))
                second_index = np.random.randint(0,len(paragraph_2))

                c = "[CLS] " + paragraph_1[first_index]
                s = "[CLS] " + paragraph_2[second_index]
                df = pd.concat([pd.DataFrame([[c,s,0]], columns=df.columns), df], ignore_index=True)
        except:
            continue
    return df

In [11]:
def process_paragraph(paragraphs):
    filtered_paragraphs = [paragraph for paragraph in paragraphs if count_sentences(paragraph) > 2]
    filtered_paragraphs = [paragraph.replace("\n", " ") for paragraph in filtered_paragraphs]
    df = generate_pos_samples(filtered_paragraphs)
    df = pd.concat([generate_neg_samples(filtered_paragraphs,df.size), df], ignore_index=True)
    return df
process_paragraph(paragraphs)

,Context,Next_Sentence,is_same_paragraph
0,[CLS] Tudo isso depende da crítica.,"[CLS] Que ela apareça, convencida e resoluta, ...",0
1,[CLS] São óbvias as conseqüências de uma tal s...,[CLS] Tudo isso depende da crítica.,0
2,[CLS] O erro produzirá o erro; amortecidos os ...,[CLS] O erro produzirá o erro; amortecidos os ...,0
3,"[CLS] Com efeito, se o crítico, na manifestaçã...",[CLS] São óbvias as conseqüências de uma tal s...,0
4,"[CLS] Se esta reforma, que eu sonho, sem esper...",[CLS] O erro produzirá o erro; amortecidos os ...,0
...,...,...,...
1336,[CLS] São óbvias as conseqüências de uma tal s...,"[CLS] As musas, privadas de um farol seguro, c...",1
1337,[CLS] São óbvias as conseqüências de uma tal s...,[CLS] O poeta oscilará entre as sentenças mal ...,1
1338,[CLS] São óbvias as conseqüências de uma tal s...,[CLS] O erro produzirá o erro; amortecidos os ...,1
1339,[CLS] São óbvias as conseqüências de uma tal s...,"[CLS] As musas, privadas de um farol seguro, c...",1


In [12]:
def list_text_files(directory):
    """Recursively list all .txt files in the given directory and subdirectories."""
    text_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                text_files.append(os.path.join(root, file))
    return text_files
files = list_text_files(directory)

In [13]:
def process_file_chunk(files):
    """Process a chunk of files and return the concatenated DataFrame."""
    r = []
    for file_path in files:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            paragraphs = text.split('\n\n')
            r.append(process_paragraph(paragraphs))
    return pd.concat(r, ignore_index=True)

In [14]:
max_threads = os.cpu_count() -1
chunk_size = len(files) // max_threads

file_chunks = [files[i * chunk_size:(i + 1) * chunk_size] for i in range(max_threads)]

# If there are any remaining files, add them to the last chunk
if len(files) % max_threads != 0:
    file_chunks[-1].extend(files[max_threads * chunk_size:])

chunk_size

20

In [15]:
import concurrent.futures
with concurrent.futures.ProcessPoolExecutor(max_workers=max_threads) as executor:
    # Submit tasks for each chunk to the thread pool
    futures = [executor.submit(process_file_chunk, chunk) for chunk in file_chunks]

    # Collect the results from each thread
    results = []
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        results.append(result)
        print(len(results))

    # Concatenate all the DataFrames from the results
    final_df = pd.concat(results, ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
final_df.sample(100).head()

,Context,Next_Sentence,is_same_paragraph
393977,[CLS] A resposta foi compelir-me fortemente a ...,[CLS] Talvez alegre.,1
418499,[CLS] Flora conhecia a predição da cabocla do ...,"[CLS] Santos falara dela em tempo, apenas ocul...",1
282886,[CLS] O finado era o marido.Raquel referiu-lhe...,[CLS] Pareciam ao nosso herói; ele é que o diz...,1
49130,"[CLS] Não lhe pergunteis, por exemplo, na 'Man...","[CLS] Homero, pela mão de Vulcano, pôs naquele...",0
77673,"[CLS] Veste de preto, e o busto fica-lhe bem, ...",[CLS] Abana-se por distração; talvez olhe tamb...,1


In [ ]:
final_df.to_csv("../data/machado.csv",index=False)